In [1]:
library(tidyverse)
library(textclean)
library(readxl)

working_dir <- getwd()
setwd("../")
source("base/it-402-dc-data_initial_exploration-base.r")
setwd(working_dir)

rm(working_dir)

── Attaching packages ───────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor




In [2]:
#rm(list = ls())

In [3]:
dbConn <- dbConnect(RSQLite::SQLite(), "../sta_it_402/data/sqa-data-db.sqlite")
dbConn

<SQLiteConnection>
  Path: /Users/aba-sah/OneDrive/work/programming/devt/r/work/sta/it-402-dc/sta_it_402/data/sqa-data-db.sqlite
  Extensions: TRUE

In [4]:
dbGetQuery(dbConn,
           "SELECT * FROM sqlite_master WHERE type = 'table' AND (name = 'centre_postcodes')")

type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,centre_postcodes,centre_postcodes,169,"CREATE TABLE ""centre_postcodes"" ( `Centre` TEXT, `CentreName` TEXT, `Postcode` TEXT, `EducationAuthority` TEXT, `GeoArea` TEXT, `Country` TEXT )"


In [5]:

## https://www.gov.scot/publications/school-level-summary-statistics/

## https://www.gov.scot/publications/scottish-government-urban-rural-classification-2016/pages/2/
## https://www.nrscotland.gov.uk/statistics-and-data/geography/our-products/scottish-postcode-directory/2022-1
## https://www.nrscotland.gov.uk/files/geography/Products/postcode-bkgrd-info.pdf
## https://statistics.gov.scot/slice?dataset=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fattainment-for-all&http%3A%2F%2Fpurl.org%2Flinked-data%2Fcube%23measureType=http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fmeasure-properties%2Faverage-total-tariff-score&http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fdimension%2FattainmentDistribution=http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fconcept%2Fattainment-distribution%2Fhighest-20&http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fdimension%2Fcomparator=http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fconcept%2Fcomparator%2Freal-establishment
## https://www.opendata.nhs.scot/sl/dataset/geography-codes-and-labels/resource/967937c4-8d67-4f39-974f-fd58c4acfda53fold


### school results from Education Scotland and demographic data from gov.scot

In [6]:
datafiles_secondary_school_level_summary_statistics <-
    c("2019" = "data/secondary_school_level_summary_statistics_2019.csv",
      "2020" = "data/secondary_school_level_summary_statistics_2020.csv",
      "2021" = "data/secondary_school_level_summary_statistics_2021.csv"
      )


datafiles_cs_entries <- list.files("data/", pattern = "stat00007", full.names = TRUE, ignore.case = TRUE, recursive = TRUE)

sel_idx <- str_detect(datafiles_cs_entries, ".csv")
datafiles_cs_entries[sel_idx]

sel_idx2 <- str_detect(datafiles_cs_entries, "dictionary|csv")
datafiles_cs_entries[!sel_idx2]

[1] "data//stat00007_graded_v4.csv"   "data//stat00007_ungraded_v4.csv"
[3] "data//stat00007_unit_v4.csv"

[1] "data//stat00007.xlsx"

In [7]:
abbreviations <- c("FTE", "CS", "ASN", "ESL", "SIMD", "FSM", "P4-P7/S1-S6/SP")

In [8]:
centre_postcodes <- bind_rows(lapply(datafiles_cs_entries[sel_idx], read_csv, 
                                     col_select = matches("Centre|postcode|ea_name|geo_area"), 
                                     name_repair = str_to_title,
                                     trim_ws = TRUE
                                    )
                             ) %>%
    rename_with(snakecase::to_upper_camel_case) %>% 

    bind_rows(read_xlsx(datafiles_cs_entries[!sel_idx2], 
                            trim_ws = TRUE, 
                       ) %>% 
              select(matches("Centre|postcode|ea_name|geo_area")) %>%
              rename_with(snakecase::to_upper_camel_case)
             ) %>% 
    rename(EducationAuthority = snakecase::to_upper_camel_case("ea_name")) %>%
              

    bind_rows(read_csv("data/9414.csv", 
                       col_select = matches("Centre|postcode|(Education Authority)"), 
                       trim_ws = TRUE, 
                       skip = 1) %>%
              rename_with(snakecase::to_upper_camel_case)
             ) %>% 

    filter(rowSums(is.na(.)) != ncol(.)) %>%
    mutate(across(where(is.character), str_squish),
           across(CentreName, ~ str_replace_all(., "\\$", "")), # flags school closed
           
           across(GeoArea, ~ gsub("(\\w+), City of", "\\1 City", .)), 
           across(GeoArea, ~ gsub("(Shetland|Orkney)", "\\1 Islands", .)), 
           across(GeoArea, ~ if_else(str_detect(., "Edinburgh"), "City of Edinburgh", .)), 
           across(GeoArea, ~ if_else(str_detect(., "Clackmannan"), "Clackmannanshire", .)), 
           across(GeoArea, ~ if_else(str_detect(., "Western Isles"), "Na h-Eileanan Siar", .)), 
           across(GeoArea, ~ if_else(str_detect(., regex("Grant aided", ignore_case = TRUE)), str_to_title(.), .)), 
           across(GeoArea, ~ str_squish(str_remove(., "Council"))),
           across(GeoArea, ~ fct_relevel(., "Grant Aided", after = Inf)),
           
           across(matches("GeoArea|Authority"), ~ gsub("\\band\\b", "\\&", .)),
           across(EducationAuthority, ~ fct_relevel(., "Independent", after = Inf)),

           across(!where(is.factor), as.factor)
          ) %>%
    
    distinct() %>%
    arrange(Centre) %>%
    glimpse()

levels(centre_postcodes$EducationAuthority)
levels(centre_postcodes$GeoArea)

length(levels(centre_postcodes$EducationAuthority))
length(levels(centre_postcodes$GeoArea))

Rows: 7561 Columns: 5
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Centre_name, Postcode, Ea_name, Geo_area
dbl (1): Centre

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 11443 Columns: 5
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Centre_name, Postcode, Ea_name, Geo_area
dbl (1): Centre

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 95659 Columns: 5
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Centre_name, Postcode, Ea_name, Geo_area
dbl (1): Centre

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE

Rows: 1,350
Columns: 5
$ Centre             <fct> 1000144, 1001930, 1002732, 1002732, 1002759, 100285…
$ CentreName         <fct> "Firpark Secondary School", "Buchanan High School",…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, G67 2UF, KA13 6DE, DD3 8…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


[1] "Aberdeen City Council Education Department"        
 [2] "Aberdeenshire Council Education Department"        
 [3] "Angus Council Education Department"                
 [4] "Argyll & Bute Council Education Department"        
 [5] "City of Glasgow Council Education Department"      
 [6] "Clackmannanshire Council Education Department"     
 [7] "Dumfries & Galloway Council Education Department"  
 [8] "Dundee City Council Education Department"          
 [9] "East Ayrshire Council Education Department"        
[10] "East Dunbartonshire Council Education Department"  
[11] "East Lothian Council Education Department"         
[12] "East Renfrewshire Council Education Department"    
[13] "Falkirk Council Education Department"              
[14] "Fife Council Education Department"                 
[15] "Highland Council Education Department"             
[16] "Inverclyde Council Education Department"           
[17] "Midlothian Council Education Department"           
[18] "North Ayrshire Council Education Department"       
[19] "North Lanarkshire Council Education Department"    
[20] "Orkney Islands Council Education Department"       
[21] "Perth & Kinross Council Education Department"      
[22] "Renfrewshire Council Education Department"         
[23] "Scottish Borders Council Education Department"     
[24] "Shetland Islands Council Education Department"     
[25] "South Ayrshire Council Education Department"       
[26] "South Lanarkshire Council Education Department"    
[27] "Stirling Council Education Department"             
[28] "The City of Edinburgh Council Education Department"
[29] "The Moray Council Education Department"            
[30] "West Dunbartonshire Council Education Department"  
[31] "West Lothian Council Education Department"         
[32] "Western Isles Council Education Department"        
[33] "Independent"

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "England"             "Falkirk"             "Far East"           
[16] "Fife"                "Glasgow City"        "Highland"           
[19] "Inverclyde"          "Middle East"         "Midlothian"         
[22] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[25] "North Lanarkshire"   "Orkney Islands"      "Other"              
[28] "Perth & Kinross"     "Renfrewshire"        "Scottish Borders"   
[31] "Shetland Islands"    "South Asia"          "South Ayrshire"     
[34] "South Lanarkshire"   "Stirling"            "Wales"              
[37] "West Dunbartonshire" "West Lothian"

[1] 33

[1] 38

In [9]:
#centre_postcodes %>%
#    group_by(across(matches("Centre"))) %>%
#    add_tally() %>%
#    filter(n > 1)

centre_postcodes <- centre_postcodes %>%
    filter(!is.na(GeoArea)) %>%

    full_join(centre_postcodes %>%
                  filter(is.na(GeoArea)) %>%
                  select(- GeoArea) %>%
                  rename(EducationAuthority_fill = EducationAuthority)
    ) %>%
    mutate(across(EducationAuthority, ~ coalesce(., EducationAuthority_fill))) %>%
    select(- EducationAuthority_fill) %>%
    glimpse()


centre_postcodes %>%

    group_by(across(matches("Centre"))) %>%
    add_tally() %>%
    filter(n > 1) %>%
    arrange(Centre)

centre_postcodes %>%
    filter(if_all(c(EducationAuthority, GeoArea), is.na)) %>%
    nrow()

centre_postcodes %>%
    filter(is.na(GeoArea)) %>%
    nrow()

centre_postcodes %>%
    filter(is.na(EducationAuthority)) %>%
    nrow()

Joining, by = c("Centre", "CentreName", "Postcode")


Rows: 988
Columns: 5
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <fct> "Firpark Secondary School", "Buchanan High School",…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


Centre,CentreName,Postcode,EducationAuthority,GeoArea,n
<fct>,<fct>,<fct>,<fct>,<fct>,<int>
5251834,Banchory Academy,AB31 5UJ,Aberdeenshire Council Education Department,Aberdeenshire,2
5251834,Banchory Academy,AB31 5TQ,Aberdeenshire Council Education Department,Aberdeenshire,2


[1] 0

[1] 0

[1] 520

In [10]:
# school website and google maps both have  AB31 5UJ 
# manually deleting other

centre_postcodes %>%
    distinct(Centre, Postcode) %>%

    group_by(across(matches("Centre"))) %>%
    add_tally() %>%
    filter(n > 1)

Centre,Postcode,n
<fct>,<fct>,<int>
5251834,AB31 5UJ,2
5251834,AB31 5TQ,2


In [11]:
centre_postcodes <- centre_postcodes %>%
    filter(!((Centre == "5251834") & (Postcode == "AB31 5TQ")))

### Manually fill in remaining blanks in postcodes

In [12]:
summary_stats_schools <- bind_rows(lapply(datafiles_secondary_school_level_summary_statistics, read_csv, 
                                          col_select = matches("Authority|SeedCode|Name"),
                                          trim_ws = TRUE, 
                                          na = c("*", "#", "..", "z", "x")
                                         )
                             ) %>% 

    filter(rowSums(is.na(.)) != ncol(.)) %>%
    distinct() %>%
    rename_with(snakecase::to_upper_camel_case) %>%

    glimpse()

Rows: 358 Columns: 3
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Local Authority, School Name
dbl (1): SeedCode

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 357 Columns: 3
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Local Authority, School Name
dbl (1): SeedCode

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 357 Columns: 3
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Local Authority, School Name
dbl (1): SeedCode

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 384
Columns: 3
$ LocalAuthority <chr> "Aberdeen City", "Aberdeen City", "Aberdeen City", "Abe…
$ SeedCode       <dbl> 5244439, 5235634, 5234034, 5235839, 5243335, 5243238, 5…
$ SchoolName     <chr> "Aberdeen Grammar School", "Bridge of Don Academy", "Bu…


In [13]:
summary_stats_schools <- summary_stats_schools %>%

    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%

    mutate(across(where(is.character), str_squish),
           across(SchoolName, ~ str_replace_all(., "\\$", "")), # flags school closed
           across(SeedCode, as.character),
           
           across(LocalAuthority, ~ if_else(str_detect(., "Edinburgh"), "City of Edinburgh", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Clackmannan"), "Clackmannanshire", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Western Isles"), "Na h-Eileanan Siar", .)), 
           across(LocalAuthority, ~ gsub("\\band\\b", "\\&", .)),
           across(LocalAuthority, ~ if_else(str_detect(., regex("Grant aided", ignore_case = TRUE)), str_to_title(.), .)), 
           across(LocalAuthority, ~ str_squish(str_remove(., "Council"))),
           across(LocalAuthority, ~ fct_relevel(., "Grant Aided", after = Inf)),
           
           across(everything(), as.factor)
          ) %>%
    distinct() %>%

    glimpse()


Rows: 361
Columns: 3
$ LocalAuthority <fct> Aberdeen City, Aberdeen City, Aberdeen City, Aberdeen C…
$ SeedCode       <fct> 5244439, 5235634, 5234034, 5235839, 5243335, 5243238, 5…
$ SchoolName     <fct> Aberdeen Grammar School, Bridge of Don Academy, Bucksbu…


In [14]:
summary_stats_schools %>%
    group_by(SeedCode) %>%
    add_tally() %>%
    filter(n > 1) %>%
    arrange(SeedCode, desc(n)) 

LocalAuthority,SeedCode,SchoolName,n
<fct>,<fct>,<fct>,<int>
City of Edinburgh,5533937,Wester Hailes Education Centre,2
City of Edinburgh,5533937,Wester Hailes High School,2
Scottish Borders,5632730,Jedburgh Grammar School,2
Scottish Borders,5632730,Jedburgh Grammar Campus - Secondary,2


In [15]:
levels(summary_stats_schools$LocalAuthority)

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "Falkirk"             "Fife"                "Glasgow City"       
[16] "Highland"            "Inverclyde"          "Midlothian"         
[19] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[22] "North Lanarkshire"   "Orkney Islands"      "Perth & Kinross"    
[25] "Renfrewshire"        "Scottish Borders"    "Shetland Islands"   
[28] "South Ayrshire"      "South Lanarkshire"   "Stirling"           
[31] "West Dunbartonshire" "West Lothian"        "Grant Aided"

In [16]:
summary_stats_schools <- summary_stats_schools %>%

    left_join(centre_postcodes %>%
                  distinct(Centre, Postcode), 
              by = c("SeedCode" = "Centre")
             )

In [17]:
nrow(summary_stats_schools)

summary_stats_schools %>%
    distinct(SeedCode) %>%
    nrow()

summary_stats_schools %>%
    filter(is.na(Postcode)) %>%
    distinct(SeedCode, SchoolName) %>%
    nrow()

summary_stats_schools %>%
    filter(!is.na(Postcode)) %>%
    distinct(SeedCode, SchoolName, Postcode) %>%
    nrow()

[1] 361

[1] 359

[1] 15

[1] 346

In [18]:
summary_stats_schools %>%
    filter(is.na(Postcode))

summary_stats_schools %>%
    filter(is.na(Postcode)) %>%
    distinct(SeedCode, SchoolName, Postcode) %>%
    mutate(seed = paste0("(SeedCode == \"", SeedCode, "\") ~ \"\",", " # ", SchoolName),
           Postcode = "\"\""
          ) %>%
    distinct(seed)

LocalAuthority,SeedCode,SchoolName,Postcode
<fct>,<fct>,<fct>,<fct>
Argyll & Bute,8109931,Islay High School,NA
Argyll & Bute,8111030,Lochgilphead High School,NA
Dumfries & Galloway,5948630,Lockerbie Academy,NA
Dumfries & Galloway,5948738,Moffat Academy,NA
Highland,5130239,Ardnamurchan High School,NA
Highland,5119332,Fortrose Academy,NA
Highland,5142539,Glen Urquhart High School,NA
Highland,5142431,Kilchuimen Academy,NA
Highland,5135834,Mallaig High School,NA


seed
<chr>
"(SeedCode == ""8109931"") ~ """", # Islay High School"
"(SeedCode == ""8111030"") ~ """", # Lochgilphead High School"
"(SeedCode == ""5948630"") ~ """", # Lockerbie Academy"
"(SeedCode == ""5948738"") ~ """", # Moffat Academy"
"(SeedCode == ""5130239"") ~ """", # Ardnamurchan High School"
"(SeedCode == ""5119332"") ~ """", # Fortrose Academy"
"(SeedCode == ""5142539"") ~ """", # Glen Urquhart High School"
"(SeedCode == ""5142431"") ~ """", # Kilchuimen Academy"
"(SeedCode == ""5135834"") ~ """", # Mallaig High School"


In [19]:
tmp_df <- summary_stats_schools %>%
    filter(is.na(Postcode)) %>%
    distinct() %>%

    # where not on school website retrieved from council website, unless otherwise flagged
    mutate(across(Postcode, as.character),
           across(Postcode, ~ case_when((SeedCode == "8109931") ~ "PA43 7LS", # Islay High School -  (google has PA43 7JS)
                                        (SeedCode == "8111030") ~ "PA31 8AA", # Lochgilphead High School
                                        (SeedCode == "5948630") ~ "DG11 2AT", # Lockerbie Academy
                                        (SeedCode == "5948738") ~ "DG10 9QF", # Moffat Academy
                                        (SeedCode == "5130239") ~ "PH36 4JA", # Ardnamurchan High School
                                        (SeedCode == "5119332") ~ "IV10 8TW", # Fortrose Academy
                                        (SeedCode == "5142539") ~ "IV63 6XA", # Glen Urquhart High School
                                        (SeedCode == "5142431") ~ "PH32 4DL", # Kilchuimen Academy
                                        (SeedCode == "5135834") ~ "PH41 4RG", # Mallaig High School
                                        (SeedCode == "8517231") ~ "ML7 4DH", # Calderhead High School
                                        (SeedCode == "8506736") ~ "ML5 2HT", # St Ambrose High School
                                        (SeedCode == "6003230") ~ "SG19 1BL", # Sanday Secondary School
                                        (SeedCode == "6003834") ~ "KW17 2AE", # Stronsay Secondary School
                                        (SeedCode == "6003036") ~ "KW17 2DH", # Westray Secondary School
                                        (SeedCode == "6104134") ~ "ZE2 9AB", # Whalsay School - Secondary (from google, neither school nor coucil has school address)
                                        TRUE ~ .
                                       )),
           across(Postcode, as.factor),           
        )

tmp_df

LocalAuthority,SeedCode,SchoolName,Postcode
<fct>,<fct>,<fct>,<fct>
Argyll & Bute,8109931,Islay High School,PA43 7LS
Argyll & Bute,8111030,Lochgilphead High School,PA31 8AA
Dumfries & Galloway,5948630,Lockerbie Academy,DG11 2AT
Dumfries & Galloway,5948738,Moffat Academy,DG10 9QF
Highland,5130239,Ardnamurchan High School,PH36 4JA
Highland,5119332,Fortrose Academy,IV10 8TW
Highland,5142539,Glen Urquhart High School,IV63 6XA
Highland,5142431,Kilchuimen Academy,PH32 4DL
Highland,5135834,Mallaig High School,PH41 4RG


In [20]:
education_authorities <- centre_postcodes %>%
    rename(LocalAuthority = GeoArea) %>%
    filter(EducationAuthority != "Independent") %>%
    select(contains("Authority")) %>%
    distinct() %>%
    mutate(across(everything(), as.character)) %>%
    drop_na()

glimpse(education_authorities)

getEducationAuthority("Fife", education_authorities)
getEducationAuthority("Na h-Eileanan Siar", education_authorities)

Rows: 32
Columns: 2
$ EducationAuthority <chr> "North Lanarkshire Council Education Department", "…
$ LocalAuthority     <chr> "North Lanarkshire", "South Lanarkshire", "Glasgow …


[1] "Fife Council Education Department"

[1] "Western Isles Council Education Department"

In [21]:
# ll stte schools - run a batch fill

tmp_df$LocalAuthority

tmp_df$EducationAuthority <-
    sapply(tmp_df$LocalAuthority, 
           getEducationAuthority, 
           education_authorities)

tmp_df

[1] Argyll & Bute       Argyll & Bute       Dumfries & Galloway
 [4] Dumfries & Galloway Highland            Highland           
 [7] Highland            Highland            Highland           
[10] North Lanarkshire   North Lanarkshire   Orkney Islands     
[13] Orkney Islands      Orkney Islands      Shetland Islands   
33 Levels: Aberdeen City Aberdeenshire Angus ... Grant Aided

LocalAuthority,SeedCode,SchoolName,Postcode,EducationAuthority
<fct>,<fct>,<fct>,<fct>,<chr>
Argyll & Bute,8109931,Islay High School,PA43 7LS,Argyll & Bute Council Education Department
Argyll & Bute,8111030,Lochgilphead High School,PA31 8AA,Argyll & Bute Council Education Department
Dumfries & Galloway,5948630,Lockerbie Academy,DG11 2AT,Dumfries & Galloway Council Education Department
Dumfries & Galloway,5948738,Moffat Academy,DG10 9QF,Dumfries & Galloway Council Education Department
Highland,5130239,Ardnamurchan High School,PH36 4JA,Highland Council Education Department
Highland,5119332,Fortrose Academy,IV10 8TW,Highland Council Education Department
Highland,5142539,Glen Urquhart High School,IV63 6XA,Highland Council Education Department
Highland,5142431,Kilchuimen Academy,PH32 4DL,Highland Council Education Department
Highland,5135834,Mallaig High School,PH41 4RG,Highland Council Education Department


In [22]:
glimpse(centre_postcodes)

centre_postcodes <- centre_postcodes %>%
    anti_join(tmp_df %>%
              mutate(across(everything(), ~ fct_drop(.))) %>%
              rename(Centre = SeedCode)) %>%
    rbind(tmp_df %>%
              mutate(across(everything(), ~ fct_drop(.))) %>%
              rename_with(~ c("Centre", "CentreName", "GeoArea"), c("SeedCode", "SchoolName", "LocalAuthority"))
    ) %>%
    glimpse()

Rows: 987
Columns: 5
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <fct> "Firpark Secondary School", "Buchanan High School",…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


Joining, by = c("Centre", "Postcode", "EducationAuthority")


Rows: 1,002
Columns: 5
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <fct> "Firpark Secondary School", "Buchanan High School",…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


In [23]:
centre_postcodes %>%
    tail(20)

Centre,CentreName,Postcode,EducationAuthority,GeoArea
<fct>,<fct>,<fct>,<fct>,<fct>
8811296,Basil Paterson College,EH2 4NA,Independent,City of Edinburgh
8811793,Wallace College,EH1 1EE,NA,City of Edinburgh
9194010,System Computer Information Technology Institute (Syscoms),NA,NA,Middle East
9900080,East Coast College (Lowestoft Campus),NR32 2NB,NA,England
9994971,SQA Operations,G2 8DQ,NA,Glasgow City
8109931,Islay High School,PA43 7LS,Argyll & Bute Council Education Department,Argyll & Bute
8111030,Lochgilphead High School,PA31 8AA,Argyll & Bute Council Education Department,Argyll & Bute
5948630,Lockerbie Academy,DG11 2AT,Dumfries & Galloway Council Education Department,Dumfries & Galloway
5948738,Moffat Academy,DG10 9QF,Dumfries & Galloway Council Education Department,Dumfries & Galloway


#### END &ndash; manually fill in remaining blanks in postcodes

### Clean Geographical Metadata

In [24]:
centre_postcodes %>%
    filter(is.na(Postcode)) %>% 
    arrange(CentreName)

Centre,CentreName,Postcode,EducationAuthority,GeoArea
<fct>,<fct>,<fct>,<fct>,<fct>
3025187,AAS College Applied Arts Studies,NA,NA,Other
3018091,AAT Business Park,NA,NA,Far East
3003647,ADNOC SQA Centre,NA,NA,Middle East
3020282,Britannia International Academy,NA,NA,Middle East
3008576,Business And Technology Education Centre (BTEC),NA,NA,Middle East
3002349,Centre of Excellence for Technical and Vocational Education,NA,NA,Middle East
3017982,Myanmar Metropolitan College,NA,NA,Far East
3019500,National Company for Training and Education Technologies,NA,NA,Middle East
3017915,Racer Engineering College,NA,NA,Far East


In [25]:
levels(centre_postcodes$GeoArea)

setdiff(summary_stats_schools$LocalAuthority, centre_postcodes$GeoArea)
setdiff(centre_postcodes$GeoArea, summary_stats_schools$LocalAuthority)

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "England"             "Falkirk"             "Far East"           
[16] "Fife"                "Glasgow City"        "Highland"           
[19] "Inverclyde"          "Middle East"         "Midlothian"         
[22] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[25] "North Lanarkshire"   "Orkney Islands"      "Other"              
[28] "Perth & Kinross"     "Renfrewshire"        "Scottish Borders"   
[31] "Shetland Islands"    "South Asia"          "South Ayrshire"     
[34] "South Lanarkshire"   "Stirling"            "Wales"              
[37] "West Dunbartonshire" "West Lothian"

[1] "Grant Aided"

[1] "Middle East" "Far East"    "Wales"       "England"     "Other"      
[6] "South Asia"

In [26]:
centre_postcodes <- centre_postcodes %>%
    mutate(across(GeoArea, ~ fct_relevel(., #setdiff(centre_postcodes$GeoArea, summary_stats_schools$LocalAuthority),
                                         "England", "Wales", "Middle East", "Far East", "South Asia", "Other",
                                         after = Inf)))

levels(centre_postcodes$GeoArea)

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "Falkirk"             "Fife"                "Glasgow City"       
[16] "Highland"            "Inverclyde"          "Midlothian"         
[19] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[22] "North Lanarkshire"   "Orkney Islands"      "Perth & Kinross"    
[25] "Renfrewshire"        "Scottish Borders"    "Shetland Islands"   
[28] "South Ayrshire"      "South Lanarkshire"   "Stirling"           
[31] "West Dunbartonshire" "West Lothian"        "England"            
[34] "Wales"               "Middle East"         "Far East"           
[37] "South Asia"          "Other"

In [27]:
centre_postcodes <- centre_postcodes %>%
    mutate(Country = case_when((GeoArea %in% summary_stats_schools$LocalAuthority) ~ "Scotland",
                               (GeoArea %in% c("England", "Wales")) ~ as.character(GeoArea),
                               TRUE ~ "Other"
                              ),
           across(Country, ~ factor(., levels = c("Scotland", "England", "Wales", "Other")))
          )


centre_postcodes %>%
    distinct(GeoArea, Country) %>%
    arrange(Country, GeoArea)

GeoArea,Country
<fct>,<fct>
Aberdeen City,Scotland
Aberdeenshire,Scotland
Angus,Scotland
Argyll & Bute,Scotland
City of Edinburgh,Scotland
Clackmannanshire,Scotland
Dumfries & Galloway,Scotland
Dundee City,Scotland
East Ayrshire,Scotland


### Derive SIMD and Rural/Urban classifications - Scottish centres

### "external data" - urban/rural classification

In [28]:
sg_6_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_6_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_6_fold_urban_rural_classification)

sg_8_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_8_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_8_fold_urban_rural_classification)

Rows: 6 Columns: 3
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): URNAME, URDESC
dbl (1): URCLASS

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 6
Columns: 3
$ URCLASS <dbl> 1, 2, 3, 4, 5, 6
$ URNAME  <chr> "Large Urban Areas", "Other Urban Areas", "Accessible Small To…
$ URDESC  <chr> "Settlements of over 125 000 people", "Settlements of 10 000 t…


Rows: 8 Columns: 3
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): URNAME, URDESC
dbl (1): URCLASS

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 8
Columns: 3
$ URCLASS <dbl> 1, 2, 3, 4, 5, 6, 7, 8
$ URNAME  <chr> "Large Urban Areas", "Other Urban Areas", "Accessible Small To…
$ URDESC  <chr> "Settlements of over 125 000 people", "Settlements of 10 000 t…


In [29]:
sg_6_fold_urban_rural_classification <- sg_6_fold_urban_rural_classification %>%
    rename_with(~ c("URCode", "URClassification", "Description"))  %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural6Fold"))

sg_8_fold_urban_rural_classification <- sg_8_fold_urban_rural_classification %>%
    rename_with(~ c("URCode", "URClassification", "Description"))  %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural8Fold"))

sg_6_fold_urban_rural_classification 
sg_8_fold_urban_rural_classification 


UrbanRural6FoldCode,UrbanRural6FoldClassification,Description
<fct>,<fct>,<chr>
1,Large Urban Areas,Settlements of over 125 000 people
2,Other Urban Areas,Settlements of 10 000 to 125 000 people
3,Accessible Small Towns,Settlements of between 3 000 and 10 000 people and within a 30 minute drive time of a Settlement of 10 000 or more
4,Remote Small Towns,Settlements of between 3 000 and 10 000 people and with a drive time of over 30 minutes to a Settlement of 10 000 or more
5,Accessible Rural,Areas with a population of less than 3 000 people and within a 30 minute drive time of a Settlement of 10 000 or more
6,Remote Rural,Areas with a population of less than 3 000 people and with a drive time of over 30 minutes to a Settlement of 10 000 or more


UrbanRural8FoldCode,UrbanRural8FoldClassification,Description
<fct>,<fct>,<chr>
1,Large Urban Areas,Settlements of over 125 000 people
2,Other Urban Areas,Settlements of 10 000 to 125 000 people
3,Accessible Small Towns,Settlements of between 3 000 and 10 000 people and within a 30 minute drive time of a Settlement of 10 000 or more
4,Remote Small Towns,Settlements of between 3 000 and 10 000 people and with a drive time between 30 and 60 minutes to a Settlement of 10 000 or more
5,Very Remote Small Towns,Settlements of between 3 000 and 10 000 people and with a drive time of over 60 minutes to a Settlement of 10 000 or more
6,Accessible Rural Areas,Areas with a population of less than 3 000 people and within a drive time of 30 minutes to a Settlement of 10 000 or more
7,Remote Rural Areas,Areas with a population of less than 3 000 people and with a drive time of between 30 and 60 minutes to a Settlement of 10 000 or more
8,Very Remote Rural Areas,Areas with a population of less than 3 000 people and with a drive time of over 60 minutes to a Settlement of 10 000 or more


In [30]:
sg_2_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_2_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_2_fold_urban_rural_classification)

sg_2_fold_urban_rural_classification <- sg_2_fold_urban_rural_classification %>%
    select(- contains("Area")) %>%
    rename_with(~ c("URCode", "URClassification", "Description")) %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural2Fold"))


sg_3_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_3_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_3_fold_urban_rural_classification)

sg_3_fold_urban_rural_classification <- sg_3_fold_urban_rural_classification %>%
    select(- contains("Area")) %>%
    rename_with(~ c("URCode", "URClassification", "Description")) %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural3Fold"))

sg_2_fold_urban_rural_classification 
sg_3_fold_urban_rural_classification 



Rows: 2 Columns: 5
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): URNAME, URDESC
dbl (3): URCLASS, AreaSqKm, PercentageTotalLandArea

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 2
Columns: 5
$ URCLASS                 <dbl> 1, 2
$ URNAME                  <chr> "Urban Areas", "Rural Areas"
$ URDESC                  <chr> "", "Areas with a population less than 3,000 p…
$ AreaSqKm                <dbl> 1689.194, 76247.917
$ PercentageTotalLandArea <dbl> 2.16738, 97.83262


Rows: 3 Columns: 5
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (2): URNAME, URDESC
dbl (2): URCLASS, PercentageTotalLandArea

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 3
Columns: 5
$ URCLASS                 <dbl> 1, 2, 3
$ URNAME                  <chr> "Rest of Scotland", "Accessible Rural", "Remot…
$ URDESC                  <chr> "", "Accessible Areas are within a 30 minute d…
$ AreaSqKm                <dbl> 1689.19, 22024.82, 54223.09
$ PercentageTotalLandArea <dbl> 2.17, 28.26, 69.57


UrbanRural2FoldCode,UrbanRural2FoldClassification,Description
<fct>,<fct>,<chr>
1,Urban Areas,
2,Rural Areas,"Areas with a population less than 3,000 people"


UrbanRural3FoldCode,UrbanRural3FoldClassification,Description
<fct>,<fct>,<chr>
1,Rest of Scotland,
2,Accessible Rural,"Accessible Areas are within a 30 minute drive time from the centre of a Settlement of 10,000 or more."
3,Remote Rural,"Remote Areas have a drive time which is greater than 30 minutes from the centre of a Settlement of 10,000 or more."


In [31]:
levels(sg_2_fold_urban_rural_classification$UrbanRural2FoldCode)
levels(sg_2_fold_urban_rural_classification$UrbanRural2FoldClassification)

levels(sg_3_fold_urban_rural_classification$UrbanRural3FoldCode)
levels(sg_3_fold_urban_rural_classification$UrbanRural3FoldClassification)

[1] "1" "2"

[1] "Urban Areas" "Rural Areas"

[1] "1" "2" "3"

[1] "Rest of Scotland" "Accessible Rural" "Remote Rural"

In [32]:
sg_urban_rural_classification_postcode_lookup <- read_csv("data/sg_urban_rural_classification/sg_2016_urban_rural_classification_postcode_lookup.csv", 
                                                          trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_urban_rural_classification_postcode_lookup)

Rows: 235880 Columns: 11
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (7): POSTCODE, INT_DATE, DEL_DATE, TYPE, LINKPC, SPLIT_IND, CA_CODE
dbl (4): UR2FOLD, UR3FOLD, UR6FOLD, UR8FOLD

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 235,880
Columns: 11
$ POSTCODE  <chr> "AB1 0AA", "AB1 0AB", "AB1 0AD", "AB1 0AE", "AB1 0AF", "AB1 …
$ INT_DATE  <chr> "01/01/1980", "01/08/1973", "01/08/1973", "01/02/1994", "01/…
$ DEL_DATE  <chr> "01/06/1996", "01/04/1996", "01/04/1996", "01/04/1996", "01/…
$ TYPE      <chr> "0", "0", "0", "0", "L", "L", "0", "0", "L", "0", "0", "0", …
$ LINKPC    <chr> "", "", "", "", "AB14 0QL", "AB14 0SB", "", "", "AB13 0AP", …
$ SPLIT_IND <chr> "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", …
$ CA_CODE   <chr> "S12000033", "S12000033", "S12000033", "S12000034", "S120000…
$ UR2FOLD   <dbl> 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, …
$ UR3FOLD   <dbl> 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, …
$ UR6FOLD   <dbl> 3, 3, 5, 5, 3, 3, 3, 3, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, …
$ UR8FOLD   <dbl> 3, 3, 6, 6, 3, 3, 3, 3, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, …


In [33]:
# huge file - rather than delete on completion, filter and store subset before join

sg_urban_rural_classification_postcode_lookup <- sg_urban_rural_classification_postcode_lookup %>%

    select(matches("POSTCODE|FOLD|DEL_DATE|LINKPC") | matches("[2368]{1}")) %>%
    rename_with(~ paste0(str_replace(., "UR", "UrbanRural"), "Code"), starts_with("UR")) %>%
    rename(LinkPostcode = LINKPC) %>%
    rename_with(snakecase::to_upper_camel_case) %>%
    mutate(across(c(DelDate, LinkPostcode), ~ na_if(str_trim(.), ""))) %>%
    mutate(across(everything(), as.factor))

glimpse(sg_urban_rural_classification_postcode_lookup)

Rows: 235,880
Columns: 7
$ Postcode            <fct> AB1 0AA, AB1 0AB, AB1 0AD, AB1 0AE, AB1 0AF, AB1 0…
$ DelDate             <fct> 01/06/1996, 01/04/1996, 01/04/1996, 01/04/1996, 01…
$ LinkPostcode        <fct> NA, NA, NA, NA, AB14 0QL, AB14 0SB, NA, NA, AB13 0…
$ UrbanRural2FoldCode <fct> 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2,…
$ UrbanRural3FoldCode <fct> 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2,…
$ UrbanRural6FoldCode <fct> 3, 3, 5, 5, 3, 3, 3, 3, 5, 5, 3, 5, 5, 5, 5, 5, 5,…
$ UrbanRural8FoldCode <fct> 3, 3, 6, 6, 3, 3, 3, 3, 6, 6, 3, 6, 6, 6, 6, 6, 6,…


In [34]:
sg_urban_rural_classification_postcode_lookup %>%
    select(ends_with("FoldCode")) %>%
    distinct() %>%
    arrange(UrbanRural8FoldCode)

UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>
1,1,1,1
1,1,2,2
1,1,3,3
1,1,4,4
1,1,4,5
2,2,5,6
2,3,6,7
2,3,6,8


In [35]:
length(levels(sg_urban_rural_classification_postcode_lookup$Postcode))

sg_urban_rural_classification_postcode_lookup <- sg_urban_rural_classification_postcode_lookup %>%

    filter(is.na(DelDate)) %>%
    filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%

    bind_rows(sg_urban_rural_classification_postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DelDate)) 
    ) %>%
    bind_rows(sg_urban_rural_classification_postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DelDate)) %>%
                distinct(LinkPostcode) %>%
                rename(Postcode = LinkPostcode) %>%

                left_join(sg_urban_rural_classification_postcode_lookup) %>%
                filter(!is.na(Postcode)) 
    ) %>%

    distinct() %>%    
    mutate(across(c(Postcode, DelDate, LinkPostcode), ~ fct_drop(.))) 

length(levels(sg_urban_rural_classification_postcode_lookup$Postcode))

[1] 221376

Joining, by = "Postcode"


[1] 860

In [36]:
sg_urban_rural_classification_postcode_lookup %>%
    slice_sample(n = 4)

sg_urban_rural_classification_postcode_lookup %>%
    distinct(across(matches("UrbanRural"))) %>%
    arrange(UrbanRural8FoldCode)

Postcode,DelDate,LinkPostcode,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
G51 1PR,NA,NA,1,1,1,1
HS1 2BJ,NA,NA,1,1,4,5
DD2 2LJ,NA,NA,1,1,1,1
IV30 5SY,NA,NA,2,2,5,6


UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>
1,1,1,1
1,1,2,2
1,1,3,3
1,1,4,4
1,1,4,5
2,2,5,6
2,3,6,7
2,3,6,8
NA,NA,NA,NA


In [37]:
sg_urban_rural_classification_postcode_lookup %>%
    filter(if_all(matches("UrbanRural"), is.na)) %>%
    distinct() %>%
    arrange(UrbanRural8FoldCode)

Postcode,DelDate,LinkPostcode,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
NO LINKP,NA,NA,NA,NA,NA,NA


In [38]:
centre_postcodes <- centre_postcodes %>%
    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  filter(is.na(DelDate)) %>%
                  select(- c(DelDate, LinkPostcode))
             ) %>%
    mutate(across(Postcode, as.character), 
           across(Postcode, as.factor)) %>%

    glimpse()


centre_postcodes %>%
    count(Country) %>%
    rename(NoOfCentres = n) %>%
    
    full_join(centre_postcodes %>%
                filter(if_any(starts_with("UrbanRural"), is.na)) %>%
                count(Country) %>%
                rename(URClassificationNotSet = n)
    )

Joining, by = "Postcode"


Rows: 1,002
Columns: 10
$ Centre              <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 10029…
$ CentreName          <fct> "Firpark Secondary School", "Buchanan High School"…
$ Postcode            <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11…
$ EducationAuthority  <fct> North Lanarkshire Council Education Department, No…
$ GeoArea             <fct> North Lanarkshire, North Lanarkshire, North Lanark…
$ Country             <fct> Scotland, Scotland, Scotland, Scotland, Scotland, …
$ UrbanRural2FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural3FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural6FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1, 2, 1, 1,…
$ UrbanRural8FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 6, 2, 1, 2, 1, 1,…


Joining, by = "Country"


Country,NoOfCentres,URClassificationNotSet
<fct>,<int>,<int>
Scotland,892,28
England,24,24
Wales,2,2
Other,84,84


In [39]:
centre_postcodes %>%
    filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) 
    

Centre,CentreName,Postcode,EducationAuthority,GeoArea,Country,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1004620,North West Community Campus,DG2 9DF,Dumfries & Galloway Council Education Department,Dumfries & Galloway,Scotland,NA,NA,NA,NA
1004743,Bertha Park High School,PH1 0AU,Perth & Kinross Council Education Department,Perth & Kinross,Scotland,NA,NA,NA,NA
1005405,Lockhart Campus,KA20 3JX,North Ayrshire Council Education Department,North Ayrshire,Scotland,NA,NA,NA,NA
3000451,Rewards Training,G2 3BZ,NA,Glasgow City,Scotland,NA,NA,NA,NA
3001024,Calder Associates (Scotland) Limited,KY12 OQX,NA,Fife,Scotland,NA,NA,NA,NA
3006166,Ritchies Training Centre,G21 4AU,NA,Glasgow City,Scotland,NA,NA,NA,NA
3012743,Fitness Training Scotland,KA1 9GX,NA,East Ayrshire,Scotland,NA,NA,NA,NA
3012808,Alma Care Training,G72 6AN,NA,South Lanarkshire,Scotland,NA,NA,NA,NA
3023486,Lead Scotland,EH5 2FF,NA,City of Edinburgh,Scotland,NA,NA,NA,NA


In [40]:
sg_urban_rural_classification_postcode_lookup %>%
    filter(Postcode %in% 
              (centre_postcodes %>%
                filter(if_any(starts_with("UrbanRural"), is.na)) %>%
                distinct(Postcode) %>%
                deframe() %>%
                as.character()  
              ))

Postcode,DelDate,LinkPostcode,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
AB15 5PD,09/07/2015,NA,1,1,1,1
DD2 1NL,09/07/2015,DD2 1NJ,1,1,1,1
DG2 9DY,01/09/1995,DG2 9DX,1,1,2,2
G2 3BZ,09/07/2015,NA,1,1,1,1
G21 4AU,19/07/2016,G21 4AQ,1,1,1,1
G4 9LD,19/10/2016,G4 9LH,1,1,1,1
G52 4RU,03/06/2008,NA,1,1,1,1
G78 2QY,17/10/2013,G78 2RA,1,1,2,2
IV17 0WA,23/01/2002,IV17 0PB,1,1,4,4


In [41]:
centre_postcodes %>%
    filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
    nrow()

centre_postcodes %>%
    filter((Country == "Scotland") & !(if_any(starts_with("UrbanRural"), is.na))) %>% 
    nrow()

centre_postcodes %>%
    filter((Country == "Scotland") & if_all(starts_with("UrbanRural"), ~ !is.na(.))) %>%
    nrow()


dim(centre_postcodes)

[1] 28

[1] 864

[1] 864

[1] 1002   10

In [42]:
centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | if_all(starts_with("UrbanRural"), ~ !is.na(.))) %>%

    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
                select(- starts_with("UrbanRural")) %>%

                left_join(centre_postcodes %>%
                            filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
                            distinct(Postcode) %>%

                            left_join(sg_urban_rural_classification_postcode_lookup %>%
                                          filter(!is.na(DelDate)) %>%
                                          select(Postcode, LinkPostcode)) %>%

                            left_join(sg_urban_rural_classification_postcode_lookup %>%
                                          filter(is.na(DelDate)) %>%
                                          select(Postcode, matches("UrbanRural")),
                                      by = c("LinkPostcode" = "Postcode")) 
                          )
                 ) %>%
    glimpse()

length(levels(centre_postcodes$Postcode))


centre_postcodes %>%
    count(Country) %>%
    rename(NoOfCentres = n) %>%
    
    full_join(centre_postcodes %>%
                filter(if_any(starts_with("UrbanRural"), is.na)) %>%
                count(Country) %>%
                rename(URClassificationNotSet = n)
    )

Joining, by = "Postcode"
Joining, by = "Postcode"


Rows: 1,002
Columns: 11
$ Centre              <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 10029…
$ CentreName          <fct> "Firpark Secondary School", "Buchanan High School"…
$ Postcode            <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11…
$ EducationAuthority  <fct> North Lanarkshire Council Education Department, No…
$ GeoArea             <fct> North Lanarkshire, North Lanarkshire, North Lanark…
$ Country             <fct> Scotland, Scotland, Scotland, Scotland, Scotland, …
$ UrbanRural2FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural3FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural6FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1, 2, 1, 1,…
$ UrbanRural8FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 6, 2, 1, 2, 1, 1,…
$ LinkPostcode        <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…


[1] 965

Joining, by = "Country"


Country,NoOfCentres,URClassificationNotSet
<fct>,<int>,<int>
Scotland,892,18
England,24,24
Wales,2,2
Other,84,84


In [43]:

centre_postcodes %>%
    distinct(across(matches("UrbanRural\\dFoldCode"))) %>%
    arrange(UrbanRural8FoldCode)

UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>
1,1,1,1
1,1,2,2
1,1,3,3
1,1,4,4
1,1,4,5
2,2,5,6
2,3,6,7
2,3,6,8
NA,NA,NA,NA


In [44]:
postcode_lookup <- read_csv("data/postcode_lookup/SmallUser.csv", trim_ws = TRUE, na = c("", "NA")) %>% 
                                filter(rowSums(is.na(.)) != ncol(.)) %>%

    bind_rows(read_csv("data/postcode_lookup/LargeUser.csv", trim_ws = TRUE, na = c("", "NA")) %>% 
                                    filter(rowSums(is.na(.)) != ncol(.))) %>%

    glimpse()

Rows: 192894 Columns: 57
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (44): Postcode, PostcodeDistrict, PostcodeSector, DateOfIntroduction, Da...
dbl (13): GridReferenceEasting, GridReferenceNorthing, Latitude, Longitude, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 49663 Columns: 51
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (44): Postcode, PostcodeDistrict, PostcodeSector, DateOfIntroduction, Da...
dbl  (7): GridReferenceEasting, GridReferenceNorthing, Latitude, Longitude, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 242,557
Columns: 58
$ Postcode                                   <chr> "G31 2LZ", "G31 2NF", "G31 …
$ PostcodeDistrict                           <chr> "G31", "G31", "G31", "G31",…
$ PostcodeSector                             <chr> "G31 2", "G31 2", "G31 2", …
$ DateOfIntroduction                         <chr> "1/8/1973 00:00:00", "1/12/…
$ DateOfDeletion                             <chr> NA, "15/4/2010 00:00:00", N…
$ GridReferenceEasting                       <dbl> 261052, 261015, 261096, 261…
$ GridReferenceNorthing                      <dbl> 665316, 665076, 665491, 665…
$ Latitude                                   <dbl> 55.86095, 55.85878, 55.8625…
$ Longitude                                  <dbl> -4.221647, -4.222114, -4.22…
$ SplitIndicator                             <chr> "N", "N", "N", "N", "N", "N…
$ CouncilArea2019Code                        <chr> "S12000049", "S12000049", "…
$ UKParliamentaryConstituency2005Code        <chr> "S14000032", "S14000032", "…
$ ScottishParl

In [45]:
range(postcode_lookup$ScottishIndexOfMultipleDeprivation2020Rank)

[1] "0001" "6976"

In [46]:
postcode_lookup <- postcode_lookup %>%
    rename(SIMD2020Rank = ScottishIndexOfMultipleDeprivation2020Rank)

simd2020Rank_range <- range(postcode_lookup$SIMD2020Rank)


postcode_lookup <- postcode_lookup %>%
    select(Postcode, DateOfDeletion, LinkedSmallUserPostcode, contains(c("CouncilArea2019", "Settlement2016", "SIMD", "UrbanRural"))) %>%
    distinct() %>%
    mutate(across(where(is.character), ~ str_squish(.)),
           across(c(DateOfDeletion, LinkedSmallUserPostcode), ~ na_if(., "")),
           across(where(is.numeric), as.integer), 
           across(matches("20\\d{2}Code"), as.factor)) %>%
    glimpse()


postcode_lookup %>%
    head()
postcode_lookup %>%
    tail()


Rows: 242,540
Columns: 8
$ Postcode                <chr> "G31 2LZ", "G31 2NF", "G31 2NW", "G31 2NX", "G…
$ DateOfDeletion          <chr> NA, "15/4/2010 00:00:00", NA, NA, NA, NA, NA, …
$ LinkedSmallUserPostcode <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ CouncilArea2019Code     <fct> S12000049, S12000049, S12000049, S12000049, S1…
$ Settlement2016Code      <fct> S20001250, S20001250, S20001250, S20001250, S2…
$ SIMD2020Rank            <chr> "3012", "3172", "3012", "3887", "3665", "3665"…
$ UrbanRural6Fold2016Code <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6…
$ UrbanRural8Fold2016Code <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 8, 8, 8, 8, 8…


Postcode,DateOfDeletion,LinkedSmallUserPostcode,CouncilArea2019Code,Settlement2016Code,SIMD2020Rank,UrbanRural6Fold2016Code,UrbanRural8Fold2016Code
<chr>,<chr>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>
G31 2LZ,NA,NA,S12000049,S20001250,3012,1,1
G31 2NF,15/4/2010 00:00:00,NA,S12000049,S20001250,3172,1,1
G31 2NW,NA,NA,S12000049,S20001250,3012,1,1
G31 2NX,NA,NA,S12000049,S20001250,3887,1,1
G31 2NY,NA,NA,S12000049,S20001250,3665,1,1
G31 2NZ,NA,NA,S12000049,S20001250,3665,1,1


Postcode,DateOfDeletion,LinkedSmallUserPostcode,CouncilArea2019Code,Settlement2016Code,SIMD2020Rank,UrbanRural6Fold2016Code,UrbanRural8Fold2016Code
<chr>,<chr>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>
ZE2 9WX,11/4/2008 00:00:00,NO LINKP,S12000027,S20001334,4698,4,5
ZE2 9WY,4/7/2007 00:00:00,NO LINKP,S12000027,S20001334,4698,4,5
ZE2 9YL,11/4/2019 00:00:00,NO LINKP,S12000027,NA,3814,6,8
ZE2 9YN,3/5/2011 00:00:00,NO LINKP,S12000027,S20001334,4698,4,5
ZE2 9YP,9/7/2004 00:00:00,NO LINKP,S12000027,NA,4509,6,8
ZE2 9YQ,25/5/2006 00:00:00,NO LINKP,S12000027,S20001334,4698,4,5


In [47]:
postcode_lookup <- postcode_lookup %>%

    filter(is.na(DateOfDeletion)) %>%
    filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%

    bind_rows(postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DateOfDeletion)) 
    ) %>%
    bind_rows(postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DateOfDeletion)) %>%
                distinct(LinkedSmallUserPostcode) %>%
                rename(Postcode = LinkedSmallUserPostcode) %>%

                left_join(postcode_lookup) %>%
                filter(!is.na(Postcode)) 
    ) %>%
        
    distinct() %>%
    glimpse()

Joining, by = "Postcode"


Rows: 964
Columns: 8
$ Postcode                <chr> "AB11 6DY", "AB11 6ET", "AB11 8HD", "AB12 3AL"…
$ DateOfDeletion          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ LinkedSmallUserPostcode <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ CouncilArea2019Code     <fct> S12000033, S12000033, S12000033, S12000033, S1…
$ Settlement2016Code      <fct> S20001020, S20001020, S20001020, S20001020, S2…
$ SIMD2020Rank            <chr> "2720", "2720", "1300", "2241", "5522", "2241"…
$ UrbanRural6Fold2016Code <fct> 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 5, 1, 1, 1, 1, 1…
$ UrbanRural8Fold2016Code <fct> 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 6, 1, 1, 1, 1, 1…


In [48]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    filter(!(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe()))) %>%
    select(EducationAuthority, Centre, CentreName, Postcode, starts_with("UrbanRural")) %>%
    distinct()

EducationAuthority,Centre,CentreName,Postcode,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
NA,3009084,Ageas Retail Ltd,G71 5PU,1,1,2,2
NA,4196775,Forward Training Partnership,EH54 6NG,1,1,2,2
Perth & Kinross Council Education Department,5349834,Breadalbane Academy,PH15 2DU,2,3,6,7
Independent,5780136,Beaconhurst School,FK9 4RR,1,1,2,2
Renfrewshire Council Education Department,8631344,Kersland School,PA2 7BU,1,1,1,1
NA,3001024,Calder Associates (Scotland) Limited,KY12 OQX,NA,NA,NA,NA
NA,3006166,Ritchies Training Centre,G21 4AU,1,1,1,1
NA,3012743,Fitness Training Scotland,KA1 9GX,NA,NA,NA,NA
NA,4197437,Training Matters (Scotland) Limited,G52 4RU,NA,NA,NA,NA


In [49]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    filter(!(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe()))) %>%
    select(EducationAuthority, Centre, CentreName, Postcode, starts_with("UrbanRural")) %>%
    distinct() %>%

    left_join(postcode_lookup %>%
                  filter(LinkedSmallUserPostcode != "NO LINKP") %>%
                  select(Postcode, DateOfDeletion, LinkedSmallUserPostcode) 
    ) %>%

    left_join(postcode_lookup %>%
                  select(Postcode, contains(c("CouncilArea2019", "Settlement2016", "SIMD2020", "UrbanRural"))),
              by = c("LinkedSmallUserPostcode" = "Postcode")
    )

Joining, by = "Postcode"


EducationAuthority,Centre,CentreName,Postcode,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode,DateOfDeletion,LinkedSmallUserPostcode,CouncilArea2019Code,Settlement2016Code,SIMD2020Rank,UrbanRural6Fold2016Code,UrbanRural8Fold2016Code
<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>
NA,3009084,Ageas Retail Ltd,G71 5PU,1,1,2,2,29/10/2018 00:00:00,G71 5PD,S12000050,S20001383,4374,2,2
NA,4196775,Forward Training Partnership,EH54 6NG,1,1,2,2,NA,NA,NA,NA,NA,NA,NA
Perth & Kinross Council Education Department,5349834,Breadalbane Academy,PH15 2DU,2,3,6,7,NA,NA,NA,NA,NA,NA,NA
Independent,5780136,Beaconhurst School,FK9 4RR,1,1,2,2,24/7/2019 00:00:00,FK9 4RS,S12000030,S20001477,6921,2,2
Renfrewshire Council Education Department,8631344,Kersland School,PA2 7BU,1,1,1,1,NA,NA,NA,NA,NA,NA,NA
NA,3001024,Calder Associates (Scotland) Limited,KY12 OQX,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,3006166,Ritchies Training Centre,G21 4AU,1,1,1,1,19/7/2016 00:00:00,G21 4AQ,S12000049,S20001250,0678,1,1
NA,3006166,Ritchies Training Centre,G21 4AU,1,1,1,1,19/7/2016 00:00:00,G21 4AQ,S12000049,S20001250,0678,1,1
NA,3012743,Fitness Training Scotland,KA1 9GX,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [50]:
centre_postcodes %>%
    filter((Country == "Scotland") & if_all(starts_with("UrbanRural"), is.na)) %>%
    select(contains(c("Authority", "Centre")), Postcode) %>%
    distinct() %>%
    arrange(EducationAuthority)

EducationAuthority,Centre,CentreName,Postcode
<fct>,<fct>,<fct>,<fct>
Dumfries & Galloway Council Education Department,1004620,North West Community Campus,DG2 9DF
Dumfries & Galloway Council Education Department,5940745,Langlands School,DG2 9DF
Midlothian Council Education Department,5546036,Newbattle High School,EH22 4SX
North Ayrshire Council Education Department,1005405,Lockhart Campus,KA20 3JX
North Ayrshire Council Education Department,8215839,Largs Academy,KA30 9DR
Orkney Islands Council Education Department,6003230,Sanday Secondary School,SG19 1BL
Perth & Kinross Council Education Department,1004743,Bertha Park High School,PH1 0AU
West Lothian Council Education Department,5502330,West Calder High School,EH55 8BF
West Lothian Council Education Department,5511844,Cedarbank School,EH54 6FW


In [51]:
# fill in remaining blanks... 

centre_postcodes %>%
    filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
    select(- starts_with("UrbanRural")) %>%
    
    filter(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe())) %>%

    left_join(postcode_lookup %>%
                  select(Postcode, contains(c("CouncilArea2019", "Settlement2016", "IndexOfMultipleDeprivation", "UrbanRural")))
             ) %>%
    rename_with(~ str_replace(., "2016", ""), starts_with("UrbanRural")) %>%
    

    left_join(sg_6_fold_urban_rural_classification %>%
                  select(- Description)) %>%
    left_join(sg_8_fold_urban_rural_classification %>%
                 select(- Description)) %>%

    select(EducationAuthority, Centre, CentreName, Postcode, contains(c("CouncilArea2019", "Settlement2016", "IndexOfMultipleDeprivation", "UrbanRural"))) 


Joining, by = "Postcode"
Joining, by = "UrbanRural6FoldCode"
Joining, by = "UrbanRural8FoldCode"


EducationAuthority,Centre,CentreName,Postcode,CouncilArea2019Code,Settlement2016Code,UrbanRural6FoldCode,UrbanRural8FoldCode,UrbanRural6FoldClassification,UrbanRural8FoldClassification
<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Dumfries & Galloway Council Education Department,1004620,North West Community Campus,DG2 9DF,S12000006,S20001177,2,2,Other Urban Areas,Other Urban Areas
Perth & Kinross Council Education Department,1004743,Bertha Park High School,PH1 0AU,S12000048,NA,5,6,Accessible Rural,Accessible Rural Areas
North Ayrshire Council Education Department,1005405,Lockhart Campus,KA20 3JX,S12000021,S20001453,2,2,Other Urban Areas,Other Urban Areas
NA,3000451,Rewards Training,G2 3BZ,S12000049,S20001250,1,1,Large Urban Areas,Large Urban Areas
NA,3000451,Rewards Training,G2 3BZ,S12000049,S20001250,1,1,Large Urban Areas,Large Urban Areas
NA,3012808,Alma Care Training,G72 6AN,S12000029,S20001250,1,1,Large Urban Areas,Large Urban Areas
NA,3023486,Lead Scotland,EH5 2FF,S12000036,S20001198,1,1,Large Urban Areas,Large Urban Areas
West Lothian Council Education Department,5502330,West Calder High School,EH55 8BF,S12000040,NA,5,6,Accessible Rural,Accessible Rural Areas
Midlothian Council Education Department,5546036,Newbattle High School,EH22 4SX,S12000019,S20001082,2,2,Other Urban Areas,Other Urban Areas


In [52]:
postcode_lookup %>%
    anti_join(sg_urban_rural_classification_postcode_lookup) %>%
    rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  distinct(across(matches("^UrbanRural\\d"))) %>%
                  drop_na()
             ) %>%
    distinct(across(matches("^(UrbanRural\\dFold)"))) %>%
    relocate(UrbanRural2FoldCode, UrbanRural3FoldCode)

sg_urban_rural_classification_postcode_lookup %>%
    distinct(across(matches("^(UrbanRural\\dFold)")))


Joining, by = "Postcode"
Joining, by = c("UrbanRural6FoldCode", "UrbanRural8FoldCode")


UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>
1,1,2,2
1,1,1,1
2,2,5,6


UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>
1,1,1,1
1,1,3,3
2,2,5,6
1,1,2,2
2,3,6,7
1,1,4,4
1,1,4,5
2,3,6,8
NA,NA,NA,NA


In [53]:
postcode_lookup %>%
    anti_join(sg_urban_rural_classification_postcode_lookup) %>%
    rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  distinct(across(matches("^UrbanRural\\d"))) %>%
                  drop_na()
             ) %>%
    relocate(UrbanRural2FoldCode, UrbanRural3FoldCode, .before = UrbanRural6FoldCode)

Joining, by = "Postcode"
Joining, by = c("UrbanRural6FoldCode", "UrbanRural8FoldCode")


Postcode,DateOfDeletion,LinkedSmallUserPostcode,CouncilArea2019Code,Settlement2016Code,SIMD2020Rank,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<chr>,<chr>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>
DG2 9DF,NA,NA,S12000006,S20001177,0455,1,1,2,2
EH22 4SX,NA,NA,S12000019,S20001082,5532,1,1,2,2
EH5 2FF,NA,NA,S12000036,S20001198,6707,1,1,1,1
EH55 8BF,NA,NA,S12000040,NA,3691,2,2,5,6
G72 6AN,NA,NA,S12000029,S20001250,5336,1,1,1,1
KA20 3JX,NA,NA,S12000021,S20001453,2407,1,1,2,2
KA30 9DR,NA,NA,S12000021,NA,3129,2,2,5,6
PH1 0AU,NA,NA,S12000048,NA,4624,2,2,5,6
FK9 4RS,NA,NA,S12000030,S20001477,6921,1,1,2,2


In [54]:
postcode_lookup %>%
    filter(Postcode == "G2 3BZ") 

Postcode,DateOfDeletion,LinkedSmallUserPostcode,CouncilArea2019Code,Settlement2016Code,SIMD2020Rank,UrbanRural6Fold2016Code,UrbanRural8Fold2016Code
<chr>,<chr>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>
G2 3BZ,NA,NA,S12000049,S20001250,2198,1,1
G2 3BZ,9/7/2015 00:00:00,NA,S12000049,S20001250,2198,1,1


In [55]:
tmp_df <- centre_postcodes %>%
    filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na))

tmp_df <- tmp_df %>%
    select(- starts_with("UrbanRural")) %>%

    filter(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe())) %>%

    left_join(postcode_lookup %>%
                  filter(is.na(DateOfDeletion)) %>%
                  select(Postcode, LinkedSmallUserPostcode, matches("^UrbanRural\\d"))
             ) %>%
    rename_with(~ str_replace(., "2016", ""), matches("^UrbanRural\\d")) %>%

    rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  distinct(across(matches("^UrbanRural\\d"))) %>%
                  drop_na()
             ) 

tmp_df %>%
    select(matches("^(Centre|Postcode|UrbanRural)"))

centre_postcodes %>%
    filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na)) %>%

    anti_join(tmp_df, by = "Centre"
             ) %>%
    select(matches("^(Centre|Postcode|UrbanRural)"))

centre_postcodes %>%
    filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na))  %>%
    select(- matches("UrbanRural|DateOfDeletion")) %>%

    inner_join(postcode_lookup %>%
                   filter(!is.na(DateOfDeletion)) %>%
                   distinct()
             ) 

Joining, by = "Postcode"
Joining, by = c("UrbanRural6FoldCode", "UrbanRural8FoldCode")


Centre,CentreName,Postcode,UrbanRural6FoldCode,UrbanRural8FoldCode,UrbanRural2FoldCode,UrbanRural3FoldCode
<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>
1004620,North West Community Campus,DG2 9DF,2,2,1,1
1004743,Bertha Park High School,PH1 0AU,5,6,2,2
1005405,Lockhart Campus,KA20 3JX,2,2,1,1
3000451,Rewards Training,G2 3BZ,1,1,1,1
3012808,Alma Care Training,G72 6AN,1,1,1,1
3023486,Lead Scotland,EH5 2FF,1,1,1,1
5502330,West Calder High School,EH55 8BF,5,6,2,2
5546036,Newbattle High School,EH22 4SX,2,2,1,1
5592631,Workers Educational Association,EH5 2FF,1,1,1,1


Centre,CentreName,Postcode,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
3001024,Calder Associates (Scotland) Limited,KY12 OQX,NA,NA,NA,NA
3012743,Fitness Training Scotland,KA1 9GX,NA,NA,NA,NA
4197437,Training Matters (Scotland) Limited,G52 4RU,NA,NA,NA,NA
5294738,Aberdeen City Council,AB15 5PD,NA,NA,NA,NA
5511844,Cedarbank School,EH54 6FW,NA,NA,NA,NA
5590183,Midlothian Council - Communities and Lifelong Learning,EH22 3ZG,NA,NA,NA,NA
6003230,Sanday Secondary School,SG19 1BL,NA,NA,NA,NA


Joining, by = "Postcode"


Centre,CentreName,Postcode,EducationAuthority,GeoArea,Country,LinkPostcode,DateOfDeletion,LinkedSmallUserPostcode,CouncilArea2019Code,Settlement2016Code,SIMD2020Rank,UrbanRural6Fold2016Code,UrbanRural8Fold2016Code
<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>
3000451,Rewards Training,G2 3BZ,NA,Glasgow City,Scotland,NA,9/7/2015 00:00:00,NA,S12000049,S20001250,2198,1,1
4197437,Training Matters (Scotland) Limited,G52 4RU,NA,Glasgow City,Scotland,NA,3/6/2008 00:00:00,NA,S12000049,S20001250,1610,1,1
5294738,Aberdeen City Council,AB15 5PD,NA,Aberdeen City,Scotland,NA,9/7/2015 00:00:00,NA,S12000033,S20001020,4724,1,1


In [56]:
tmp_df %>%
    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na)) %>%
                select(- starts_with("UrbanRural")) %>%

                anti_join(tmp_df, by = "Centre"
                         ) %>%
                
                left_join(postcode_lookup %>%
                              filter(!is.na(DateOfDeletion)) %>%
                              select(Postcode, LinkedSmallUserPostcode)) %>%

                left_join(postcode_lookup %>%
                              filter(is.na(DateOfDeletion)) %>%
                              select(Postcode, matches("UrbanRural")),
                          by = c("LinkedSmallUserPostcode" = "Postcode")
                         ) %>% 
              
                rename_with(~ str_replace(., "2016", ""), matches("^UrbanRural\\d")) %>%

                rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

                left_join(sg_urban_rural_classification_postcode_lookup %>%
                              distinct(across(matches("^UrbanRural\\d"))) %>%
                              drop_na()
                         ) 
                ) 

Joining, by = "Postcode"
Joining, by = c("UrbanRural6FoldCode", "UrbanRural8FoldCode")


Centre,CentreName,Postcode,EducationAuthority,GeoArea,Country,LinkPostcode,LinkedSmallUserPostcode,UrbanRural6FoldCode,UrbanRural8FoldCode,UrbanRural2FoldCode,UrbanRural3FoldCode
<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>
1004620,North West Community Campus,DG2 9DF,Dumfries & Galloway Council Education Department,Dumfries & Galloway,Scotland,NA,NA,2,2,1,1
1004743,Bertha Park High School,PH1 0AU,Perth & Kinross Council Education Department,Perth & Kinross,Scotland,NA,NA,5,6,2,2
1005405,Lockhart Campus,KA20 3JX,North Ayrshire Council Education Department,North Ayrshire,Scotland,NA,NA,2,2,1,1
3000451,Rewards Training,G2 3BZ,NA,Glasgow City,Scotland,NA,NA,1,1,1,1
3012808,Alma Care Training,G72 6AN,NA,South Lanarkshire,Scotland,NA,NA,1,1,1,1
3023486,Lead Scotland,EH5 2FF,NA,City of Edinburgh,Scotland,NA,NA,1,1,1,1
5502330,West Calder High School,EH55 8BF,West Lothian Council Education Department,West Lothian,Scotland,NA,NA,5,6,2,2
5546036,Newbattle High School,EH22 4SX,Midlothian Council Education Department,Midlothian,Scotland,NA,NA,2,2,1,1
5592631,Workers Educational Association,EH5 2FF,NA,City of Edinburgh,Scotland,NA,NA,1,1,1,1


In [57]:
# fill in remaining blanks... 

centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | if_all(starts_with("UrbanRural"), ~ !is.na(.))) %>%

    rbind(tmp_df %>%
                bind_rows(centre_postcodes %>%
                            filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na)) %>%
                            select(- starts_with("UrbanRural")) %>%

                            anti_join(tmp_df, by = "Centre"
                                     ) %>%

                            left_join(postcode_lookup %>%
                                          filter(!is.na(DateOfDeletion)) %>%
                                          select(Postcode, LinkedSmallUserPostcode)) %>%

                            left_join(postcode_lookup %>%
                                          filter(is.na(DateOfDeletion)) %>%
                                          select(Postcode, matches("UrbanRural")),
                                      by = c("LinkedSmallUserPostcode" = "Postcode")
                                     ) %>% 

                            rename_with(~ str_replace(., "2016", ""), matches("^UrbanRural\\d")) %>%

                            rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

                            left_join(sg_urban_rural_classification_postcode_lookup %>%
                                          distinct(across(matches("^UrbanRural\\d"))) %>%
                                          drop_na()
                                     ) 
                            ) %>%
                select(names(centre_postcodes)) 
              
             ) %>%

    glimpse()

Joining, by = "Postcode"
Joining, by = c("UrbanRural6FoldCode", "UrbanRural8FoldCode")


Rows: 1,002
Columns: 11
$ Centre              <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 10029…
$ CentreName          <fct> "Firpark Secondary School", "Buchanan High School"…
$ Postcode            <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11…
$ EducationAuthority  <fct> North Lanarkshire Council Education Department, No…
$ GeoArea             <fct> North Lanarkshire, North Lanarkshire, North Lanark…
$ Country             <fct> Scotland, Scotland, Scotland, Scotland, Scotland, …
$ UrbanRural2FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural3FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural6FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1, 2, 1, 1,…
$ UrbanRural8FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 6, 2, 1, 2, 1, 1,…
$ LinkPostcode        <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…


#### END &ndash; "external data" &ndash; urban/rural classification

### SIMD
* using the Scottish Index of Multiple Deprivation 2020 rankings

In [58]:
postcode_lookup %>%
    select(Postcode, SIMD2020Rank, DateOfDeletion, LinkedSmallUserPostcode) %>%
    group_by(Postcode, SIMD2020Rank) %>%
    add_tally() %>%
    filter(n > 1) %>%
    #arrange(Postcode, desc(DateOfDeletion), SIMD2020Rank)
    distinct(Postcode) %>%
    nrow()

postcode_lookup %>%
    select(Postcode, SIMD2020Rank, DateOfDeletion, LinkedSmallUserPostcode) %>%
    group_by(Postcode, SIMD2020Rank, DateOfDeletion) %>%
    add_tally() %>%
    filter(n > 1)

[1] 71

Postcode,SIMD2020Rank,DateOfDeletion,LinkedSmallUserPostcode,n
<chr>,<chr>,<chr>,<chr>,<int>


In [59]:
centre_postcodes <- centre_postcodes %>%
    left_join(postcode_lookup %>%
                  filter(is.na(DateOfDeletion)) %>%
                  select(Postcode, SIMD2020Rank) 
             )

centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
    distinct(EducationAuthority, Centre, CentreName, Postcode)

Joining, by = "Postcode"


Centre,CentreName,Postcode,EducationAuthority
<fct>,<fct>,<chr>,<fct>
3009084,Ageas Retail Ltd,G71 5PU,NA
4196775,Forward Training Partnership,EH54 6NG,NA
5349834,Breadalbane Academy,PH15 2DU,Perth & Kinross Council Education Department
5780136,Beaconhurst School,FK9 4RR,Independent
8631344,Kersland School,PA2 7BU,Renfrewshire Council Education Department
3006166,Ritchies Training Centre,G21 4AU,NA
5135532,Kinlochleven High School,PH50 4QE,Highland Council Education Department
5406331,Buckhaven High School,KY8 1HL,Fife Council Education Department
5939534,North West Community Campus,DG2 9DY,Dumfries & Galloway Council Education Department


In [60]:
tmp_df <- centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
    select(- SIMD2020Rank) %>%

    left_join(postcode_lookup %>%
                  filter(!is.na(DateOfDeletion) & (LinkedSmallUserPostcode != "NO LINKP")) %>%
                  distinct(Postcode, LinkedSmallUserPostcode) 
             ) %>%

    left_join(postcode_lookup %>%
                  distinct(Postcode, SIMD2020Rank),
              by = c("LinkedSmallUserPostcode" = "Postcode")
    )

# remaining matches not in use anywhere else; use (deleted ranking)?
#tmp_df%>%
#    filter(!is.na(SIMD2020Rank)) %>%
#
#    bind_rows(tmp_df %>%
#
#                filter(is.na(SIMD2020Rank)) %>%
#                select(- SIMD2020Rank) %>%
#
#                left_join(postcode_lookup %>%
#                              select(Postcode, SIMD2020Rank) 
#                         )
#              )

Joining, by = "Postcode"


In [61]:
glimpse(centre_postcodes)

Rows: 1,002
Columns: 12
$ Centre              <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 10029…
$ CentreName          <fct> "Firpark Secondary School", "Buchanan High School"…
$ Postcode            <chr> "ML1 2PR", "ML5 2HT", "G67 2UF", "KA13 6DE", "DD3 …
$ EducationAuthority  <fct> North Lanarkshire Council Education Department, No…
$ GeoArea             <fct> North Lanarkshire, North Lanarkshire, North Lanark…
$ Country             <fct> Scotland, Scotland, Scotland, Scotland, Scotland, …
$ UrbanRural2FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural3FoldCode <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,…
$ UrbanRural6FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1, 2, 1, 1,…
$ UrbanRural8FoldCode <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 6, 2, 1, 2, 1, 1,…
$ LinkPostcode        <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ SIMD2020Rank        <chr> "6660", "0860", "1738", "0499", "6028", "2531", "2…


In [62]:
centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | !is.na(SIMD2020Rank)) %>%
 
    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
                select(- SIMD2020Rank) %>%

                left_join(postcode_lookup %>%
                              filter(!is.na(DateOfDeletion) & (LinkedSmallUserPostcode != "NO LINKP")) %>%
                              distinct(Postcode, LinkedSmallUserPostcode) 
                         ) %>%

                left_join(postcode_lookup %>%
                              distinct(Postcode, SIMD2020Rank),
                          by = c("LinkedSmallUserPostcode" = "Postcode")
                )
             ) %>%
    relocate(SIMD2020Rank, .after = last_col()) %>%
    glimpse()


centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank))

Joining, by = "Postcode"


Rows: 1,002
Columns: 13
$ Centre                  <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 1…
$ CentreName              <fct> "Firpark Secondary School", "Buchanan High Sch…
$ Postcode                <chr> "ML1 2PR", "ML5 2HT", "G67 2UF", "KA13 6DE", "…
$ EducationAuthority      <fct> North Lanarkshire Council Education Department…
$ GeoArea                 <fct> North Lanarkshire, North Lanarkshire, North La…
$ Country                 <fct> Scotland, Scotland, Scotland, Scotland, Scotla…
$ UrbanRural2FoldCode     <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1…
$ UrbanRural3FoldCode     <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1…
$ UrbanRural6FoldCode     <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1, 2, 1…
$ UrbanRural8FoldCode     <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 6, 2, 1, 2, 1…
$ LinkPostcode            <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ LinkedSmallUserPostcode <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ SIMD2020Rank  

Centre,CentreName,Postcode,EducationAuthority,GeoArea,Country,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode,LinkPostcode,LinkedSmallUserPostcode,SIMD2020Rank
<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>
4196775,Forward Training Partnership,EH54 6NG,NA,West Lothian,Scotland,1,1,2,2,NA,NA,NA
5349834,Breadalbane Academy,PH15 2DU,Perth & Kinross Council Education Department,Perth & Kinross,Scotland,2,3,6,7,NA,NA,NA
8631344,Kersland School,PA2 7BU,Renfrewshire Council Education Department,Renfrewshire,Scotland,1,1,1,1,NA,NA,NA
3001024,Calder Associates (Scotland) Limited,KY12 OQX,NA,Fife,Scotland,NA,NA,NA,NA,NA,NA,NA
3012743,Fitness Training Scotland,KA1 9GX,NA,East Ayrshire,Scotland,NA,NA,NA,NA,NA,NA,NA
4197437,Training Matters (Scotland) Limited,G52 4RU,NA,Glasgow City,Scotland,NA,NA,NA,NA,NA,NA,NA
5294738,Aberdeen City Council,AB15 5PD,NA,Aberdeen City,Scotland,NA,NA,NA,NA,NA,NA,NA
5511844,Cedarbank School,EH54 6FW,West Lothian Council Education Department,West Lothian,Scotland,NA,NA,NA,NA,NA,NA,NA
5590183,Midlothian Council - Communities and Lifelong Learning,EH22 3ZG,NA,Midlothian,Scotland,NA,NA,NA,NA,NA,NA,NA


In [ ]:
***

In [84]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    filter(is.na(SIMD2020Rank)) %>%
    distinct(Postcode, across(matches("Centre|UrbanRural8FoldCode|SIMD2020"))) %>%

    select(- SIMD2020Rank) %>%
    left_join(postcode_lookup %>%
                  distinct(Postcode, DateOfDeletion, SIMD2020Rank) 
             ) 

Joining, by = "Postcode"


Centre,CentreName,Postcode,UrbanRural8FoldCode,SIMD2020RelativeRank,SIMD2020Quintile,DateOfDeletion,SIMD2020Rank
<fct>,<fct>,<chr>,<fct>,<int>,<int>,<chr>,<chr>
4196775,Forward Training Partnership,EH54 6NG,2,NA,NA,17/7/2020 00:00:00,2131
5349834,Breadalbane Academy,PH15 2DU,7,NA,NA,28/4/2021 00:00:00,4192
8631344,Kersland School,PA2 7BU,1,NA,NA,31/7/2018 00:00:00,5744
3001024,Calder Associates (Scotland) Limited,KY12 OQX,NA,NA,NA,NA,NA
3012743,Fitness Training Scotland,KA1 9GX,NA,NA,NA,NA,NA
4197437,Training Matters (Scotland) Limited,G52 4RU,NA,NA,NA,3/6/2008 00:00:00,1610
5294738,Aberdeen City Council,AB15 5PD,NA,NA,NA,9/7/2015 00:00:00,4724
5511844,Cedarbank School,EH54 6FW,NA,NA,NA,NA,NA
5590183,Midlothian Council - Communities and Lifelong Learning,EH22 3ZG,NA,NA,NA,NA,NA


In [ ]:
***

In [63]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    
    left_join(centre_postcodes %>%
                  filter(Country == "Scotland") %>%
                  arrange(SIMD2020Rank) %>%
                  mutate(SIMD2020RelativeRank = #row_number(),
                          rank(SIMD2020Rank, ties.method = "min", na.last = "keep"),
                        ) 
         ) %>%
    #relocate(SIMD2020Rank, .before = SIMD2020RelativeRank) 
    distinct(Postcode, across(matches("Centre|SIMD2020"))) %>%
    arrange(SIMD2020Rank)


Joining, by = c("Centre", "CentreName", "Postcode", "EducationAuthority", "GeoArea", "Country", "UrbanRural2FoldCode", "UrbanRural3FoldCode", "UrbanRural6FoldCode", "UrbanRural8FoldCode", "LinkPostcode", "LinkedSmallUserPostcode", "SIMD2020Rank")


Centre,CentreName,Postcode,SIMD2020Rank,SIMD2020RelativeRank
<fct>,<fct>,<chr>,<chr>,<int>
3003191,Inverclyde Council,PA15 1HU,0001,1
8380040,Mirren Park School,PA3 1NS,0004,2
3022749,Essentials Plus,KA21 5HN,0023,3
8294933,Galloway Training Limited,KA21 5DT,0023,3
8438331,St Margaret Mary's Secondary School,G45 9NJ,0027,5
8442843,St Oswald's School,G45 9NJ,0027,5
5532434,Craigroyston Community High School,EH4 4NL,0029,7
8443742,Newhills School,G33 4HJ,0046,8
4199774,Craigend Resource Centre,PA15 2HD,0057,9


In [64]:
dim(postcode_lookup)

range(postcode_lookup$SIMD2020Rank, na.rm = TRUE) # filtered
simd2020Rank_range
quantile(as.integer(range(simd2020Rank_range)), probs = seq(0, 1, 1/5))

[1] 964   8

[1] "0001" "6966"

[1] "0001" "6976"

0%  20%  40%  60%  80% 100% 
   1 1396 2791 4186 5581 6976

In [65]:
simd2020_quintiles <- enframe(quantile(as.integer(simd2020Rank_range), probs = seq(0, 1, 1/5))) %>%
    rename_with(~ c("quintile", "rank")) %>%
    mutate(across(quintile, ~ str_remove(., "%")), 
           across(quintile, as.integer), 
           across(quintile, ~ (. / 20)), 
           across(everything(), as.integer)
          ) %>%
    filter(quintile > 0)

simd2020_quintiles

quintile,rank
<int>,<int>
1,1395
2,2790
3,4186
4,5581
5,6976


In [66]:
getSimdQuintile <-
    function(value, quintiles) {
        
        if (is.na(value))
            return(NA)
        
        row_number <- min(which(value <= deframe(quintiles["rank"])))
        return(deframe(quintiles["quintile"])[[row_number]])
    }

#getSimdQuintile <- Vectorize(getSimdQuintile)

In [67]:
getSimdQuintile(6976, simd2020_quintiles)
getSimdQuintile(NA, simd2020_quintiles)

sapply(c(1743, 3336, 5367, 2530, 6146, 0027, 0023), getSimdQuintile, simd2020_quintiles)

[1] 5

[1] NA

[1] 2 3 4 2 5 1 1

In [68]:
centre_postcodes$SIMD2020Quintile <-
    sapply(centre_postcodes$SIMD2020Rank, 
           getSimdQuintile, 
           simd2020_quintiles)

head(centre_postcodes$SIMD2020Quintile, 15)

6660 0860 1738 0499 6028 2531 2414 0516 1202 1339 1036 4258 0733 2152 4461 
   5    1    2    1    5    2    2    1    1    1    1    4    1    2    4

In [70]:
centre_postcodes <- centre_postcodes %>%
    
    left_join(centre_postcodes %>%
                  filter(Country == "Scotland") %>%
                  arrange(SIMD2020Rank) %>%
                  mutate(SIMD2020RelativeRank = #row_number(),
                          rank(SIMD2020Rank, ties.method = "min", na.last = "keep"),
                        ) 
         ) %>%
    relocate(SIMD2020Rank, SIMD2020RelativeRank, .before = SIMD2020Quintile) %>%
    glimpse()

centre_postcodes %>%
    group_by(SIMD2020Quintile) %>%
    slice_sample(n = 2)

Joining, by = c("Centre", "CentreName", "Postcode", "EducationAuthority", "GeoArea", "Country", "UrbanRural2FoldCode", "UrbanRural3FoldCode", "UrbanRural6FoldCode", "UrbanRural8FoldCode", "LinkPostcode", "LinkedSmallUserPostcode", "SIMD2020Rank", "SIMD2020Quintile")


Rows: 1,002
Columns: 15
$ Centre                  <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 1…
$ CentreName              <fct> "Firpark Secondary School", "Buchanan High Sch…
$ Postcode                <chr> "ML1 2PR", "ML5 2HT", "G67 2UF", "KA13 6DE", "…
$ EducationAuthority      <fct> North Lanarkshire Council Education Department…
$ GeoArea                 <fct> North Lanarkshire, North Lanarkshire, North La…
$ Country                 <fct> Scotland, Scotland, Scotland, Scotland, Scotla…
$ UrbanRural2FoldCode     <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1…
$ UrbanRural3FoldCode     <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1…
$ UrbanRural6FoldCode     <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1, 2, 1…
$ UrbanRural8FoldCode     <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 6, 2, 1, 2, 1…
$ LinkPostcode            <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ LinkedSmallUserPostcode <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ SIMD2020Rank  

Centre,CentreName,Postcode,EducationAuthority,GeoArea,Country,UrbanRural2FoldCode,UrbanRural3FoldCode,UrbanRural6FoldCode,UrbanRural8FoldCode,LinkPostcode,LinkedSmallUserPostcode,SIMD2020Rank,SIMD2020RelativeRank,SIMD2020Quintile
<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<int>,<int>
8490732,MI Technologies Limited,G51 4RY,NA,Glasgow City,Scotland,1,1,1,1,NA,NA,0903,115,1
4194640,Canongate Youth Project Limited,EH1 1LT,NA,City of Edinburgh,Scotland,1,1,1,1,NA,NA,1059,132,1
3005151,Azilo Training Ltd,EH21 6RH,NA,East Lothian,Scotland,1,1,1,1,NA,NA,2471,320,2
3003604,North Lanarkshire Council T/a Logintolearn,ML2 7LP,NA,North Lanarkshire,Scotland,1,1,2,2,NA,NA,2604,347,2
3024105,Training Bees,DG11 1NP,NA,Dumfries & Galloway,Scotland,2,2,5,6,NA,NA,3807,493,3
3023664,BB Training Academy,EH54 6AX,NA,West Lothian,Scotland,1,1,2,2,NA,NA,3330,438,3
3009084,Ageas Retail Ltd,G71 5PU,NA,South Lanarkshire,Scotland,1,1,2,2,NA,G71 5PD,4374,570,4
5370035,The New School Butterstone,PH2 9BT,Independent,Perth & Kinross,Scotland,2,2,5,6,NA,NA,5226,695,4
8337632,Kirkintilloch High School,G66 3PA,East Dunbartonshire Council Education Department,East Dunbartonshire,Scotland,1,1,2,2,NA,NA,5649,742,5


#### END &ndash; SIMD Ranking

### Write to DataStore

In [75]:
dbGetQuery(dbConn, "SELECT COUNT(*) FROM centre_postcodes")
dbGetQuery(dbConn, "SELECT * FROM centre_postcodes LIMIT 6")

centre_postcodes %>%
    select(- matches("Link")) %>%

    left_join(sg_2_fold_urban_rural_classification %>%
                              select(- Description)) %>%
    left_join(sg_3_fold_urban_rural_classification %>%
                              select(- Description)) %>%
    left_join(sg_6_fold_urban_rural_classification %>%
                              select(- Description)) %>%
    left_join(sg_8_fold_urban_rural_classification %>%
                              select(- Description)) %>%
                              
    relocate(UrbanRural2FoldCode, UrbanRural2FoldClassification,
             UrbanRural3FoldCode, UrbanRural3FoldClassification,
             UrbanRural6FoldCode, UrbanRural6FoldClassification,
             UrbanRural8FoldCode, UrbanRural8FoldClassification, .before = SIMD2020Rank) %>%
    glimpse()

COUNT(*)
<int>
1002


Centre,CentreName,Postcode,EducationAuthority,GeoArea,Country
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1000144,Firpark Secondary School,ML1 2PR,North Lanarkshire Council Education Department,North Lanarkshire,Scotland
1001930,Buchanan High School,ML5 2HT,North Lanarkshire Council Education Department,North Lanarkshire,Scotland
1002732,Cumbernauld Academy,G67 2UF,North Lanarkshire Council Education Department,North Lanarkshire,Scotland
1002759,Ayrshire College Kilwinning Campus,KA13 6DE,NA,North Ayrshire,Scotland
1002856,Dundee and Angus College,DD3 8LE,NA,Dundee City,Scotland
1002953,Edinburgh College,EH11 4DE,NA,City of Edinburgh,Scotland


Joining, by = "UrbanRural2FoldCode"
Joining, by = "UrbanRural3FoldCode"
Joining, by = "UrbanRural6FoldCode"
Joining, by = "UrbanRural8FoldCode"


Rows: 1,002
Columns: 17
$ Centre                        <fct> 1000144, 1001930, 1002732, 1002759, 1002…
$ CentreName                    <fct> "Firpark Secondary School", "Buchanan Hi…
$ Postcode                      <chr> "ML1 2PR", "ML5 2HT", "G67 2UF", "KA13 6…
$ EducationAuthority            <fct> North Lanarkshire Council Education Depa…
$ GeoArea                       <fct> North Lanarkshire, North Lanarkshire, No…
$ Country                       <fct> Scotland, Scotland, Scotland, Scotland, …
$ UrbanRural2FoldCode           <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1…
$ UrbanRural2FoldClassification <fct> Urban Areas, Urban Areas, Urban Areas, U…
$ UrbanRural3FoldCode           <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1…
$ UrbanRural3FoldClassification <fct> Rest of Scotland, Rest of Scotland, Rest…
$ UrbanRural6FoldCode           <fct> 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 5, 2, 1…
$ UrbanRural6FoldClassification <fct> Other Urban Areas, Other Urban Areas, Ot…
$ UrbanRural8Fol

In [78]:
writeToDataStore(centre_postcodes %>%
                    select(- matches("Link")) %>%

                    left_join(sg_2_fold_urban_rural_classification %>%
                                              select(- Description)) %>%
                    left_join(sg_3_fold_urban_rural_classification %>%
                                              select(- Description)) %>%
                    left_join(sg_6_fold_urban_rural_classification %>%
                                              select(- Description)) %>%
                    left_join(sg_8_fold_urban_rural_classification %>%
                                              select(- Description)) %>%

                    relocate(UrbanRural2FoldCode, UrbanRural2FoldClassification,
                             UrbanRural3FoldCode, UrbanRural3FoldClassification,
                             UrbanRural6FoldCode, UrbanRural6FoldClassification,
                             UrbanRural8FoldCode, UrbanRural8FoldClassification, .before = SIMD2020Rank), 
                 dbConn, "centre_postcodes", TRUE)
                 
dbGetQuery(dbConn, "SELECT COUNT(*) FROM centre_postcodes")
dbGetQuery(dbConn, "SELECT DISTINCT GeoArea FROM centre_postcodes")

Joining, by = "UrbanRural2FoldCode"
Joining, by = "UrbanRural3FoldCode"
Joining, by = "UrbanRural6FoldCode"
Joining, by = "UrbanRural8FoldCode"


COUNT(*)
<int>
1002


GeoArea
<chr>
North Lanarkshire
North Ayrshire
Dundee City
City of Edinburgh
South Lanarkshire
Renfrewshire
Glasgow City
Fife
West Lothian


In [ ]:
dbDisconnect(dbConn)